<a href="https://colab.research.google.com/github/alecbidaran/Movie-Recommender/blob/main/Api_movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2022-12-24 12:07:18--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  --.-KB/s    in 0.1s    

2022-12-24 12:07:18 (6.72 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [2]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [3]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz

--2022-12-24 12:08:51--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.237.133.81, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.13M  65.0MB/s    in 0.2s    

2022-12-24 12:08:51 (65.0 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13770165/13770165]

ngrok


In [4]:
! ./ngrok config add-authtoken 2CLFKlNXhILl6GjGnae95RlJtxy_6x5EeH2NkzLaMAYyVb7vj

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [5]:
import numpy as np 
import torch 
import pandas as pd 
from torch.autograd import Variable

In [6]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
movies_list=pd.read_csv("/content/ml-latest-small/movies.csv")
rating_list=pd.read_csv("/content/ml-latest-small/ratings.csv")
movie_list=movies_list.loc[movies_list['movieId'].isin(rating_list['movieId'].values)]
pivot=rating_list.pivot(index='movieId',columns='userId',values='rating').fillna(0)
movie_list

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [7]:
movies=movie_list.loc[movie_list['title'].str.contains("Spider-Man")].title.values
movies

array(['Spider-Man (2002)', 'Spider-Man 2 (2004)', 'Spider-Man 3 (2007)',
       'Amazing Spider-Man, The (2012)',
       'The Amazing Spider-Man 2 (2014)',
       'Untitled Spider-Man Reboot (2017)'], dtype=object)

In [8]:
pivot

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
csr=csr_matrix(pivot)
knn=NearestNeighbors(n_neighbors=5,metric='cosine')
knn.fit(csr)


NearestNeighbors(metric='cosine')

In [10]:
hashmap = {
            movie: i+1 for i, movie in
            enumerate(list(movie_list.set_index('movieId').title)) # noqa
        }
revese_hasmap={
            i+1: movie for i, movie in
            enumerate(list(movie_list.set_index('movieId').title)) }

In [11]:
def check_movies(movie):
    movies=movie_list.loc[movie_list['title'].str.contains(movie)].title.values
    return movies


In [12]:
def recommend_movies(movie):
  id=hashmap[movie]
  value=csr_matrix(pivot.loc[id])
  dis,indices=knn.kneighbors(value,n_neighbors=5)
  recommended_list=[]
  for index in indices[0]:
    recommended_list.append(revese_hasmap[index+1])
  return recommended_list


In [13]:
movies=check_movies('Star Wars')
recom=recommend_movies(movies[0])
print(recom)

['Disclosure (1994)', 'Outbreak (1995)', 'Clear and Present Danger (1994)', 'Cliffhanger (1993)', 'Net, The (1995)']


In [ ]:
movies

array(['Star Wars: Episode IV - A New Hope (1977)',
       'Star Wars: Episode V - The Empire Strikes Back (1980)',
       'Star Wars: Episode VI - Return of the Jedi (1983)',
       'Star Wars: Episode I - The Phantom Menace (1999)',
       'Star Wars: Episode II - Attack of the Clones (2002)',
       'Star Wars: Episode III - Revenge of the Sith (2005)',
       'Star Wars: The Clone Wars (2008)',
       "Empire of Dreams: The Story of the 'Star Wars' Trilogy (2004)",
       'Star Wars: Episode VII - The Force Awakens (2015)',
       'The Star Wars Holiday Special (1978)',
       'Rogue One: A Star Wars Story (2016)',
       'Star Wars: The Last Jedi (2017)',
       'Solo: A Star Wars Story (2018)'], dtype=object)

In [14]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
from flask import Flask,request,jsonify
from flask_ngrok import run_with_ngrok

In [16]:
app=Flask(__name__)
run_with_ngrok(app)
@app.route("/",methods=['GET'])
def greeting():
  return "hello every one"
@app.route("/recommend_movie",methods=["GET"])
def get_movies():
  movie_name=request.args.get('movie_name')
  movies=check_movies(movie_name)
  movie_list=recommend_movies(movies[0])
  return jsonify({"recommend_movies":movie_list})



In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://73ae-35-245-115-230.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
